In [ ]:
import star_functions as nana
import pandas as pd
import glob
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import ascii
import numpy as np
from astropy import units as u
import matplotlib.ticker as ticker
import importlib
importlib.reload(nana)
%reload_ext autoreload
%autoreload 2

In [ ]:
#og KIC 520 star star
lc, delta_f, sampling_time, exptime520 = nana.star('KIC 5202905')
# print("delta_f:", delta_f)
# print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit520, flux_fit520, weight_fit520 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs520, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit520, flux_fit520, weight_fit520, exptime520)

# plt.plot(freq_full, power_full, 'k.')
# plt.scatter(final_freqs520, refined_power, color = 'red', marker = 'o')
# plt.show()

# plt.plot(freq_mini, power_mini, 'g-')
# plt.semilogy()
# plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit520)
delta_chi2s = nana.null_chi_squared(flux_fit520, weight_fit520) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
#520 star stats
all, half, quartiles, eighths = nana.coherence_all(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
amp_change, phase_change, sigma_lnA, sigma_phi4, sigma_phij = nana.sampling_stats(all, quartiles, eighths)


# three phase values: rate_of_phase, phase_change, sigma_phi4, sigma_phij
fig, axes = plt.subplots(3, 4, figsize=(16, 9))

plt.suptitle("KIC 5202905: Poster Child", fontsize = 18)

for idx, (ax, points1, points2, points3, points4, p1, p2, p3) in enumerate(zip(axes.flat, all, half, quartiles, eighths, rate_of_phase, 
                                                                                   sigma_phi4, sigma_phij)):
    
    a_all = points1[0]  
    b_all = points1[1]  

    a_half = points2[:, 0]  
    b_half = points2[:, 1]  

    a_quarter = points3[:, 0]  
    b_quarter = points3[:, 1] 

    a_eighth = points4[:, 0]
    b_eighth= points4[:, 1]

    ax.scatter(a_half, b_half, color='orange', marker='*')
    ax.scatter(a_quarter, b_quarter, color='blue', marker='+')
    ax.scatter(a_eighth, b_eighth, color='k', marker='.')
    ax.scatter(a_all, b_all, color='red', marker='x')
    
    #phase labeling
    ax.text(0.05, 0.6, f"phase change(2): {p1:0.8e}" , transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    ax.text(0.05, 0.53, f"sigma phi(4): {p2:0.8e}", transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    ax.text(0.05, 0.46, f"sigma phi(jack): {p3:0.8e}", transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    
    ax.legend()
    ax.axvline(0, color = 'k')
    ax.axhline(0, color = 'k')
    ax.set_title(f"{final_freqs520[idx]:0.8f}")
    ax.set_xlabel("a points")
    ax.set_ylabel("b points")
    ax.grid(True)
    ax.ticklabel_format(style='sci', scilimits=(-3, 3), axis='both')
    ax.axis('equal')

plt.tight_layout()


In [ ]:
#ANOTHER POSTER CHILD KIC 7767699

lc, delta_f, sampling_time, exptime776= nana.star('KIC 7767699')
# print("delta_f:", delta_f)
# print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit776, flux_fit776, weight_fit776 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs776, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit520, flux_fit520, weight_fit520, exptime520)

# plt.plot(freq_full, power_full, 'k-')
# plt.scatter(final_freqs776, refined_power, color = 'red', marker = 'o')
# plt.show()

# plt.plot(freq_mini, power_mini, 'g-')
# plt.semilogy()
# plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit776, flux_fit776, weight_fit776, final_freqs776, exptime776)
rate_of_phase776, rate_of_amp776 = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit776)
delta_chi2s = nana.null_chi_squared(flux_fit776, weight_fit776) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
#776 star stats
all, half, quartiles, eighths = nana.coherence_all(t_fit776, flux_fit776, weight_fit776, final_freqs776, exptime776)
amp_change, phase_change, sigma_lnA, sigma_phi4, sigma_phij = nana.sampling_stats(all, quartiles, eighths)


# three phase values: rate_of_phase, phase_change, sigma_phi4, sigma_phij
fig, axes = plt.subplots(3, 4, figsize=(16, 9))

plt.suptitle("KIC 7767699: Another Poster Child", fontsize = 18)

for idx, (ax, points1, points2, points3, points4, p1, p2, p3) in enumerate(zip(axes.flat, all, half, quartiles, eighths, rate_of_phase, 
                                                                                   sigma_phi4, sigma_phij)):
    
    a_all = points1[0]  
    b_all = points1[1]  

    a_half = points2[:, 0]  
    b_half = points2[:, 1]  

    a_quarter = points3[:, 0]  
    b_quarter = points3[:, 1] 

    a_eighth = points4[:, 0]
    b_eighth= points4[:, 1]

    ax.scatter(a_half, b_half, color='orange', marker='*')
    ax.scatter(a_quarter, b_quarter, color='blue', marker='+')
    ax.scatter(a_eighth, b_eighth, color='k', marker='.')
    ax.scatter(a_all, b_all, color='red', marker='x')
    
    #phase labeling
    ax.text(0.05, 0.95, f"phase change(2): {p1:0.8e}" , transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    ax.text(0.05, 0.88, f"sigma phi(4): {p2:0.8e}", transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    ax.text(0.05, 0.81, f"sigma phi(jack): {p3:0.8e}", transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    
    ax.legend()
    ax.axvline(0, color = 'k')
    ax.axhline(0, color = 'k')
    ax.set_title(f"{final_freqs776[idx]:0.8f}")
    ax.set_xlabel("a points")
    ax.set_ylabel("b points")
    ax.grid(True)
    ax.ticklabel_format(style='sci', scilimits=(-3, 3), axis='both')
    ax.axis('equal')

plt.tight_layout()


In [ ]:
#KIC 1573876 rando star
lc, delta_f, sampling_time, exptime157 = nana.star('KIC 1573876')
# print("delta_f:", delta_f)
# print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit157, flux_fit157, weight_fit157 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs157, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit157, flux_fit157, weight_fit157, exptime157)

plt.plot(freq_full, power_full, 'k.')
plt.scatter(final_freqs157, refined_power, color = 'red', marker = 'o')
plt.title("KIC 001573876: random star")
plt.show()

plt.plot(freq_mini, power_mini, 'g-')
plt.semilogy()
plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit157, flux_fit157, weight_fit157, final_freqs157, exptime157)
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit157)
delta_chi2s = nana.null_chi_squared(flux_fit157, weight_fit157) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
#rando star stats
all, half, quartiles, eighths = nana.coherence_all(t_fit157, flux_fit157, weight_fit157, final_freqs157, exptime157)
amp_change, phase_change, sigma_lnA, sigma_phi4, sigma_phij = nana.sampling_stats(all, quartiles, eighths)


# three phase values: rate_of_phase, phase_change, sigma_phi4, sigma_phij
fig, axes = plt.subplots(3, 4, figsize=(16, 9))

plt.suptitle("KIC 001573876: Random Star", fontsize = 18)

for idx, (ax, points1, points2, points3, points4, p1, p2, p3) in enumerate(zip(axes.flat, all, half, quartiles, eighths, rate_of_phase, 
                                                                                   sigma_phi4, sigma_phij)):
    
    a_all = points1[0]  
    b_all = points1[1]  

    a_half = points2[:, 0]  
    b_half = points2[:, 1]  

    a_quarter = points3[:, 0]  
    b_quarter = points3[:, 1] 

    a_eighth = points4[:, 0]
    b_eighth= points4[:, 1]

    ax.scatter(a_half, b_half, color='orange', marker='*')
    ax.scatter(a_quarter, b_quarter, color='blue', marker='+')
    ax.scatter(a_eighth, b_eighth, color='k', marker='.')
    ax.scatter(a_all, b_all, color='red', marker='x')
    
    #phase labeling
    ax.text(0.05, 0.95, f"phase change(2): {p1:0.8e}" , transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    ax.text(0.05, 0.88, f"sigma phi(4): {p2:0.8e}", transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    ax.text(0.05, 0.81, f"sigma phi(jack): {p3:0.8e}", transform=ax.transAxes, fontsize=8,
        verticalalignment='top', color='green')
    
    ax.legend()
    ax.axvline(0, color = 'k')
    ax.axhline(0, color = 'k')
    ax.set_title(f"{final_freqs157[idx]:0.8f}")
    ax.set_xlabel("a points")
    ax.set_ylabel("b points")
    ax.grid(True)
    ax.ticklabel_format(style='sci', scilimits=(-3, 3), axis='both')
    ax.axis('equal')

plt.tight_layout()